In [1]:
from dataclasses import dataclass
import torch
from tqdm.notebook import trange
from tradezoo.agent import DecisionBatch, LogNormalBatch, ObservationBatch
from tradezoo.game import Game, Client, SineWave, Trader
from tradezoo.market import Account, Market
from tradezoo.plots import balance_plot, trades_plot

In [2]:
@dataclass(frozen=True)
class MockAgent:
    discount_factor: float

    def decide(self, observation_batch: ObservationBatch) -> DecisionBatch:
        return DecisionBatch(
            mid_price=LogNormalBatch(
                underlying_means=torch.tensor([0], dtype=torch.float),
                underlying_stds=torch.tensor([0.01], dtype=torch.float),
            ),
            spread=LogNormalBatch(
                underlying_means=torch.tensor([-3], dtype=torch.float),
                underlying_stds=torch.tensor([0.01], dtype=torch.float),
            ),
        )
    
    def evaluate(self, observation_batch: ObservationBatch) -> torch.Tensor:
        return torch.tensor(
            [observation.cash_balance + observation.asset_balance for observation in observation_batch.observations],
            dtype=torch.float
        ) / (1 - self.discount_factor)

agent = MockAgent(discount_factor=0.99)

In [3]:
trader_account = Account(cash_balance=256, asset_balance=256)
client_account = Account(cash_balance=float("inf"), asset_balance=float("inf"))
price_process = 1 + SineWave(period=256) * 0.2
trader = Trader(
    agent=agent,
    account=trader_account,
    client=Client(
        account=client_account,
        for_account=trader_account,
        ask_process=price_process * 1.1,
        bid_process=price_process * 0.9,
    ),
)
game = Game.new(
    market=Market.from_accounts([trader_account, client_account]),
    traders=[trader],
)
turn_results = [game.turn_() for _ in trange(4096)]

  0%|          | 0/4096 [00:00<?, ?it/s]

In [4]:
trades_plot(turn_results)

In [5]:
balance_plot(turn_results)